<font size="6">Нейронные сети</font>

# Ограничения линейных моделей

Вспомним материал лекции №2, на которой мы разбирали линейные модели. Напомним, что выход линейной модели является линейной комбинацией входных признаков, к которой также добавляется смещение:

$$ \large
\begin{eqnarray*}
y & = & w_1 x_1+ w_2 x_2 + ... + w_n x_n + b\\
& = & \sum_{i=1}^n x_i w_i + b \\
& = & (\vec{x}, \vec{w}) + b
\end{eqnarray*}
$$

Данную формулу можно графически представить следующим образом:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/one_neuron_linear_model.png" width="450"></center>

Если линейная модель имеет множество выходов, то каждый выход $\large y_j$ имеет свой собственный вектор весов $\large \vec{w_j}$ и свое смещение $\large b_j$ :

$$ \large y_j = (\vec{x}, \vec{w_j}) + b_j \ \ \ \ \ \ \ j= 1 ... C,$$

где $\large \vec{w_j}=[w_{1j}, w_{2j}, ... w_{nj}]^\top$.

Тогда выход всей модели можно записать в векторно-матричном виде:

$$\large \underset{[1 \times C]}{\vec{y}} = \underset{[1 \times n]}{x} \underset{[n \times C]}{W} + \underset{[1 \times C]}{\vec{b}}$$

Здесь матрица весовых коэффициентов $\large W$ образуется путем объединения векторов весовых коэффициентов для каждого выхода, а вектор смещений $\large \vec{b}$ — соответственно путем объединения смещений всех выходов:

$$ \large
\begin{eqnarray*}
W & = & [\vec{w_1}, \vec{w_2}, ... \vec{w_C}]\\
\vec{b} & = & [b_1, b_2, ... b_c]
\end{eqnarray*}
$$

Графически это можно представить следующим образом:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/linear_classifier.png" width="450"></center>

##  ХОR — проблема

Рассмотрим такую задачу: на вход модели подаётся упорядоченный набор из двух чисел согласно таблице истинности некоторой логической функции. Задача линейного классификатора — сопоставить этим числам их класс согласно таблице. Два входных числа можно изобразить как координаты точек на плоскости, а знаком $+/-$ обозначить их истинный класс. Тогда задача классификатора — построить линию, отделяющую "плюсы" (класс 1) от "минусов" (класс 0).

- При моделировании логических функций "И" и "ИЛИ" в пространстве признаков можно легко провести прямую линию, разделяющую точки разных классов.
- При моделировании логической функции "исключающее ИЛИ" (XOR) одной линией разделить точки разных классов геометрически невозможно — точки, размеченные по таблице истинности XOR, являются **линейно неразделимыми**.

То есть линейный классификатор уже не может справиться с этой, казалось бы, простой задачей.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/xor_problem.png" width="1000"></center>

## Внутриклассовая вариативность

Вспомним рассмотренный в лекции №2 датасет из статьи [[paper] 🎓 Emotion classification in Russian: feature engineering and analysis](https://link.springer.com/chapter/10.1007/978-3-030-72610-2_10).

Он размечен по **5 классам эмоций**:
- радость (joy),
- печаль (sadness),
- злость (anger),
- неуверенность (uncertainty),
- нейтральность (neutrality).

Рассмотрим линейный классификатор, обученный на этом датасете:

- метод векторизации: TF-IDF
- модель: логистическая регрессия
- усредненное значение accuracy: $0.84$.


В линейном классификаторе каждый выход модели отвечает за свой класс. Мы можем надеяться, что в процессе обучения линейного классификатора весовые вектора разных выходов выучат некий обобщенный "шаблон" образцов данного класса. Тогда при подаче на вход нового образца один из шаблонов будет "подходить" больше других, и соответствующий выход будет получать **наибольшее значение взвешенной суммы**. Номер выхода, который имеет наибольшее значение взвешенной суммы, будет восприниматься как предсказанный класс.

Визуализируем шаблоны (векторы весовых коэффициентов), получающиеся в результате обучения линейного классификатора на датасете, размеченном по классам эмоций.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/linear_classifier_emotions.png" width="450"></center>

Для каждого класса были посчитаны свои коэффициенты регрессии. Они представляют матрицу $5 \times 1739$, где $5$ — количество классов, $1739$ — количество признаков (слов).

Ожидается, что наиболее характерным слова для каждой эмоции будут присвоены самые большие коэффициенты соответствующего класса.

Выведем значения коэффициентов для слов, которые предположительно наиболее характеры для каждого класса эмоции. Также выведем топ-5 слов с самыми большими коэффициентами.

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/coefficient_matrix.txt
!wget -q https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/feature_names.txt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

W = np.loadtxt("coefficient_matrix.txt")[1:] # load coefficients, shape (5, 1739)
X = np.loadtxt("feature_names.txt", dtype=object) # load features dictionary

labels = ['Радость', 'Нейтральность',
          'Печаль', 'Неуверенность'] # class labels in dataset
expected = ['рад', 'нормальный',
            'грустный', 'думать'] # expected typical words for each class
f, axs = plt.subplots(1, 4)

for i in range(W.shape[0]):

    order = W[i].argsort() # ascending order for coefficients
    top_Wi = W[i][order][::-1][:5] # sort and extract top-5 coefficients
    top_Wi = np.insert(top_Wi,0,
                       W[i][np.where(X == expected[i])]) # insert coefficient for expected word
    top_X = X[order][::-1][:5] # words with the highest coefficients
    top_X = np.insert(top_X, 0, expected[i]) # insert expected word
    barlist = axs[i].bar(x=[x[0] for x in zip(top_X, top_Wi)],
         height=[x[1] for x in zip(top_X, top_Wi)])
    axs[i].tick_params(labelrotation=65)
    axs[i].title.set_text(labels[i])
    barlist[0].set_color('g')

f.set_figheight(3)
f.set_figwidth(20)
plt.show()

Можно заметить, что наши предположения не подтвердились: не все слова, характерные для какой-либо эмоции, получают высокий коэффициент. Следовательно, модель не учитывает вариативность данных внутри этого класса.

Ограничение линейного классификатора состоит в том, что для каждого класса существует только один шаблон. Шаблон каждого класса будет пытаться вобрать в себя информацию обо всех объектах класса сразу. Сильная внутриклассовая вариативность будет мешать линейному классификатору запоминать разные варианты объектов одного класса, и это ограничивает точность модели.

# Многослойные нейронные сети

Одна из идей для решения проблемы внутриклассовой вариативности — модифицировать модель таким образом, чтобы у нее внутри было не по одному шаблону на каждый класс, а по несколько (скажем, 100 шаблонов вместо 10 при десяти классах). Тогда бы модель имела возможность запоминать разные объекты одного класса и далее использовать эти промежуточные шаблоны для разбиения объектов на классы.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/modeified_model_nlp.png"  width="600"></center>

Реализуем эту модель на основе линейного классификатора из лекции №2.

Применяем к выходам классификатора еще один классификатор. Будет ли данная модель работать лучше?

In [ ]:
import numpy as np


x = np.random.rand(1739)  # random text
W1 = np.random.randn(1739, 100) * 0.0001  # without bias
W2 = np.random.randn(100, 10) * 0.0001  # without bias

scores1 = np.matmul(x, W1)  # matrix multiplication, equivalent x @ W1
scores2 = np.matmul(scores1, W2)  # matrix multiplication, of the next classifier

print(f"First classifier shape: {scores1.shape}")
print(f"Second classifier shape: {scores2.shape}")

Нетрудно заметить, что последовательное применение двух классификаторов к входным данным эквивалентно применению одного классификатора с матрицей весов, равной произведению двух матриц весов классификаторов, примененных последовательно.

$$\large \text{scores}_1 = x \cdot W_1 $$

$$\large \text{scores}_2 = \text{scores}_1 \cdot W_2 = x  \cdot W_1 \cdot W_2 $$

$$\large W = W_1 \cdot W_2 $$

$$\large \text{scores}_2 = x \cdot W $$

Для того, чтобы последовательно примененные классификаторы не вырождались в один, необходимо применить нелинейность к их выходам. Например, пропустить результат применения первой матрицы весов через сигмоиду:

$$\large \sigma(s)=\frac{1}{1+e^{-s}}$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/sigmoid_function.png" width="500"></center>

In [ ]:
def sigmoid(s):
    return 1 / (1 + np.exp(-s))


# define vectorized sigmoid to implement with ndarray element-wise
sigmoid_np = np.vectorize(sigmoid)

scores1 = np.matmul(x, W1)
activations = sigmoid_np(scores1)  # values after non-linear function
scores2 = np.matmul(activations, W2)

print(f"First classifier shape: {scores1.shape}")
print(f"Activations shape: {scores1.shape}")
print(f"Second classifier shape: {scores2.shape}")

Теперь вычисления выглядят так:

$$\large \text{scores}_1 = x \cdot W_1 $$

$$\large \text{activations} = σ(\text{scores}_1) $$

$$\large \text{scores}_2 = \text{activations} \cdot W_2 =σ(x \cdot W_1) \cdot W_2$$

Такая нелинейная функция, примененная к результату линейного преобразования, называется **функцией активации**. И мы уже пользовались подобной, когда разбирали Cross-Entropy Loss (Softmax).

Приведем код в порядок:

In [ ]:
class NeuralNet:
    def __init__(self):
        self.W1 = np.random.randn(3072, 100) * 0.0001
        self.W2 = np.random.randn(100, 10) * 0.0001

    def predict(self, x):
        scores1 = np.matmul(x, self.W1)  # Linear
        activations = sigmoid_np(scores1)  # activation Sigmoid
        scores2 = np.matmul(activations, self.W2)  # Linear

        return scores2


x = np.random.rand(3072)  # image
model = NeuralNet()
scores = model.predict(x)
print(f"Model output shape: {scores.shape}")

Ядром вычислений по-прежнему является скалярное произведение входов с весовыми коэффициентами.

Таким образом вводится модель искусственного нейрона — базового элемента искусственной нейронной сети. Выходом нейрона является результат применения функции активации к взвешенной сумме входных сигналов (в общем случае с учетом смещения — "bias").


<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/neurons_output.png" width="900"></center>

Нейроны в нейронных сетях объединяют в слои и соединяют между слоями по принципу "каждый с каждым". Так получаются **многослойные полносвязные нейронные сети** (fully-connected networks).

Пример полносвязной сети с двумя скрытыми слоями:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/nn_fully_connected.png"  width="500"></center>

Как добавление в модель скрытых слоев с нелинейностями позволяет решать линейно неразделимые задачи (вроде XOR или более сложные) можно пронаблюдать [в интерактивном тренажере от TensorFlow 🎮[demo]](http://playground.tensorflow.org/#activation=linear&batchSize=10&dataset=xor&regDataset=reg-plane&learningRate=0.1&regularizationRate=0&noise=0&networkShape=&seed=0.62952&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false).

##  Веса и смещения

Веса и смещения являются настраиваемыми параметрами в нейронной сети. Они настраиваются в процессе обучения, поэтому их также называют **обучаемыми параметрами**. Рассмотрим, как различные значения обучаемых параметров влияют на работу одного нейрона. Это поможет нам получить интуицию, как добавление в модель скрытых слоев с нелинейностями наделяет её большей выразительной способностью.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/why_add_bias_example.png" width="500"></center>

Рассмотрим простой пример. На вход нейрона подаётся входное значение $x$, умноженное на вес $w$. После применения сигмоидальной функции активации, в зависимости от веса, при всевозможных значениях входа мы можем получить следующие графики при $w$, равном $0.5$, $1$ и $2$:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/sigmoid_with_different_weights.png" width="700"></center>

Можно заметить, что значение веса меняет **крутизну** итоговой сигмоидальной функции.

Но что, если требуется, чтобы при $x=5$ нейрон выдавал $0$? Изменением крутизны сигмоиды этого не добиться. Требуется дополнительный параметр — смещение.


**Смещение** (англ. bias) — это дополнительный коэффициент, прибавляющийся к взвешенной сумме входов. Наличие смещения позволяет сдвинуть функцию активации влево или вправо.



<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/add_bias_example.png" width="500"></center>

Тогда при смещениях, равных $5$, $0$ и $-5$, мы можем получить сдвинутые функции активации, что способствует лучшему обучению нейронной сети:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/sigmoid_with_different_biases.png" width="700"></center>

## Нейронная сеть как универсальный аппроксиматор

Известным теоретическим результатом в области нейронных сетей является **теорема об универсальной аппроксимации**, или [теорема Цыбенко 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0_%D0%A6%D1%8B%D0%B1%D0%B5%D0%BD%D0%BA%D0%BE). Она гласит следующее:


> Искусственная нейронная сеть прямого распространения с одним скрытым слоем и сигмоидальными функциями активации может поточечно аппроксимировать любую непрерывную функцию многих переменных с любой наперед заданной точностью.

Условиями универсальной аппроксимации являются достаточное количество нейронов скрытого слоя и правильный подбор параметров (весовых коэффициентов и смещений) нейросети.

Данный результат неконструктивен, поскольку он не говорит, *сколько необходимо скрытых нейронов* для аппроксимации той или иной функции, а также *как следует подбирать веса и смещения*. Однако он дает основания полагать, что для аппроксимации любой функции с помощью нейросети **достаточно** одного скрытого слоя с нелинейностями в виде сигмоиды.

Для того, чтобы увидеть, как из суммы сигмоидальных функций можно построить произвольную функцию, рассмотрим следующую иллюстрацию:

In [ ]:
from IPython.display import HTML
from base64 import b64encode

!wget -qN https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/universal_approximation.mp4

mp4 = open("universal_approximation.mp4", "rb").read()
data_url = f"data:video/mp4;base64,{b64encode(mp4).decode()}"
HTML(f"<video width=1000  controls><source src={data_url} type='video/mp4'></video>")

## Библиотека PyTorch

Практически вся наша работа c нейронными сетями будет осуществляться с помощью [PyTorch 🛠️[doc]](https://pytorch.org/), поэтому необходимо познакомиться с основными концептами, принципами и функциями этой библиотеки.

Лучший друг в этом, конечно же, [документация 🛠️[doc]](https://pytorch.org/docs/stable/index.html), здесь же мы разберем только основные сущности и методы.

Класс `torch.Tensor` [🛠️[doc]](https://pytorch.org/docs/stable/tensors.html#torch.Tensor) предоставляет функциональность работы с многомерными массивами.

Создание "пустого" тензора:

In [ ]:
import torch

a = torch.Tensor()

Функция создания тензора из списка:

In [ ]:
a = torch.tensor([1.1, 2.2, 3.2])
a.dtype

Явное указание типа данных:

In [ ]:
a = torch.tensor([1.1, 2.2, 3.2], dtype=torch.float64)
a.dtype

Создание двумерного тензора, заполненного единицами (для нулей `zeros`)

In [ ]:
a = torch.ones(size=(3, 2))
a.size()

Создание двумерного тензора, заполненного указанным значением


In [ ]:
a = torch.full(size=(3, 2), fill_value=3.74)
a

Транспонирование (изменение порядка осей)

In [ ]:
a = a.T
a

В библиотеке реализовано большое количество математических функций

In [ ]:
c = torch.exp(a)
print("Exponents tensor:\n", c)

c += 1
print("\nAdd 1 to tensor:\n", c)

Почти всё, что есть в NumPy, есть в PyTorch. Например, суммирование значений тензора с помощью `.sum()`:

In [ ]:
c.sum()

Перестановка, удаление и добавление пространственных измерений:

In [ ]:
a = torch.zeros((2, 5, 1, 8))
print("Original tensor size:\n", a.size())

a = a.permute(dims=(2, 0, 3, 1))  # permute dimensions
print("After permute tensor size:\n", a.size())

a = a.squeeze()  # delete dimension
print("After squzee tensor size:\n", a.size())

a = a.unsqueeze(dim=0)  # add dimension
print("After unsquzee tensor size:\n", a.size())

Преобразование `torch.Tensor` в `np.ndarray`:

In [ ]:
a.numpy()

PyTorch позволяет тензору быть представлением (view) существующего тензора. Тензор представления использует те же данные, что и его базовый тензор. Поддержка `view` позволяет избежать явного копирования данных, что позволяет экономить память.

In [ ]:
a = torch.rand(2, 8)
print("Original tensor:\n", a)

b = a.view(4, 4)
print("\nTensor after view:\n", b)

print("\nTensor b uses the same memory space as tensor a:")
id(a[0, 0]) == id(b[0, 0])

Размещение тензора на GPU:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Cuda available: {torch.cuda.is_available()} \n")

a = a.to(device)  # moving tensor to gpu
b = torch.full_like(a, 2).to(device)
c = a * b  # compute on gpu (more fast with parallel computing)
c

## Обучение нейронной сети

Процесс обучения нейронных сетей состоит из двух этапов: прямого и обратного распространения.

Во время **прямого распространения** (forward pass) мы подаем на вход нейронной сети пример данных, сигналы проходят через наши слои, на каждом слое применяется **функция активации**. В конце производится расчет значений на выходе модели $y_{pred}$, которые передаются в функцию потерь $\text{Loss}$ для сравнения с целевыми значениями $y_{true}$.

$$\large y_{pred}=\text{model}(x, 𝐖)$$

$$\large L=\text{Loss}(y_{true}, y_{pred})$$

Значение функции потерь зависит от целевых значений $y_{true}$, входных данных $x$ и параметров модели $𝐖$.

$$\large L=\text{Loss}(y_{true}, \text{model}(x, 𝐖)) = f(y_{true}, x, 𝐖)$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/forward_pass.png" width="450"></center>

На этапе **обратного распространения (backward propagation)** осуществляется подсчет **градиента** функции потерь о обучаемым параметрам $\nabla_𝐖L$. Это необходимо для **обновления** параметров модели.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/backward_pass.png" width="450"></center>

В рамках этой лекции мы подробно рассмотрим этапы прямого распространения (применение функции активации и функции потерь) и обратного распространения (метод обратного распространения ошибки).

#  Функции активации

Функции активации вводят нелинейность в нейронные сети, что позволяет аппроксимировать сложные функции. Название связано с биологической аналогией, где нейроны активируются при достижении порогового потенциала.


Перцептроны, первые нейронные сети, использовали простую пороговую функцию активации:

$$\large f(x) =
\begin{cases}
0, &\text{$x<b$} \\
1, &\text{$x\geq b$}
\end{cases}
$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/threshold_function_plot.png" width="300"></center>

## Требования к функциям активации

1. **Нелинейность:** Функции активации добавляют нелинейность, необходимую для аппроксимации сложных функций, чего нельзя достичь простой линейной моделью. Без нелинейностей нейронные сети действуют как линейные модели.

2. **Дифференцируемость:** Функции активации должны быть дифференцируемыми, чтобы применять градиентные методы оптимизации.

Рассмотрим наиболее популярные функции активации и обсудим их преимущества и недостатки.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/popular_activation_functions.png" width="700"></center>

<center><em>Источник: <a href="https://arxiv.org/pdf/1911.05187.pdf">AI in Pursuit of Happiness, Finding Only Sadness: Multi-Modal Facial Emotion Recognition Challenge</a></em></center>

## Логистическая функция

Логистическая (сигмоидальная) функция — используется в задачах бинарной классификации, в основном после выхода последнего нейрона. Позволяет определить вероятность принадлежности к одному из двух классов (0 или 1).

$$\large \sigma(x)=\frac{1}{1+e^{-x}}$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/sigmoid_function.png" width="500"></center>

В отличие от пороговой функции активации, где у нейрона было всего два состояния: "активирован" или "не активирован", с логистической функцией для нейрона возможны значения "активирован на $50\%$", "активирован на $20\%$" и так далее. Если активированы несколько нейронов, можно найти нейрон с наибольшим значением активации.

[[doc] 🛠️ Сигмоидальная функция активации в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html):
```python
torch.nn.Sigmoid()
```

In [ ]:
import torch
from torch import nn

activation = nn.Sigmoid()
input_values = torch.randn(5) * 5
activation_sig = activation(input_values)
print(f"input_values: {input_values}\nactivation_sig: {activation_sig}")

Значения логистической функции активации лежат в интервале от 0 до 1, что делает ее полезной для моделирования вероятностных величин. Значения сигмоиды можно интерпретировать как вероятности, что делает ее полезной в задачах бинарной, а также многозначной (multilabel) классификации.

##  Гиперболический тангенс

Гиперболический тангенс схож с логистической функцией. Он определяется следующей формулой:

$$\large \tanh(x)=\frac{e^x - e^{-x}}{e^x+e^{-x}}$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/tanh_function.png" width="500"></center>

Значения гиперболического тангенса лежат в интервале от $-1$ до $1$, что обеспечивает симметричность относительно нуля. Гиперболический тангенс применяют в основном в скрытых слоях нейронных сетей для моделирования нелинейных зависимостей в данных.

[[doc] 🛠️ Гиперболический тангенс в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.Tanh.html):
```python
torch.nn.Tanh()
```

In [ ]:
activation = nn.Tanh()
input_values = torch.tensor([11.1529, 4.3029, 0.5081, -3.8456, -1.9058])
activation_tanh = activation(input_values)
print(f"input_values: {input_values}\nactivation_tanh: {activation_tanh}")

##  ReLU

Часто на практике применяется функция активации ReLU. Значение данной функции равно нулю для всех отрицательных входных значений и равно входному значению, если оно неотрицательно. Название ReLU (Rectified Linear Unit), "выпрямитель", связано с электротехнической аналогией.

$$\large \text{ReLU}(x)=\max(0,x)$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/relu_function.png" width="500"></center>

[[doc] 🛠️ ReLU в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html):
```python
torch.nn.ReLU()
```

In [ ]:
activation = nn.ReLU()
input_values = torch.randn(5)
activation_relu = activation(input_values)
print(f"input_values: {input_values}\nactivation_relu: {activation_relu}")

Значения ReLU лежат в диапазоне $[0, + \infty)$. Все отрицательные значения обнуляются, а положительные значения остаются без изменений. **Проблема "умерших" нейронов**. Нейроны с отрицательными входами имеют нулевой градиент и могут остановить обновление своих весов.

Функцию активации ReLU применяют в нейронных сетях в основном в скрытых слоях. Она широко используется во многих архитектурах, доказавших свою эффективность на практике.

## Leaky ReLU

Leaky ReLU (ReLU с «утечкой») является простейшей модификацией описанной выше ReLU, призванной исправить проблему "умирания" отдельных нейронов. В отличие от ReLU, данная функция не равна константе $0$ при всех отрицательных входных значениях, а реализует в этой области линейную зависимость с небольшим угловым коэффициентом (например, с угловым коэффициентом $10^{-2}$).

$$\large \text{LeakyReLU}(x, \alpha)=\max(\alpha x,x), \ \ \ \alpha<1$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/leaky_relu_function.png" width="500"></center>

[[doc] 🛠️ Leaky ReLU в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html):
```python
torch.nn.LeakyReLU()
```

In [ ]:
activation = nn.LeakyReLU(0.01)
input_values = torch.randn(5)
activation_lrelu = activation(input_values)
print(f"input_values: {input_values}\nactivation_lrelu: {activation_lrelu}")

Значения LeakyReLU лежат в диапазоне $(-\infty, +\infty)$. Использование ненулевого наклона для отрицательных значений помогает избежать проблемы "умерших нейронов", которая возникает при использовании обычной ReLU. Однако необходимо выбирать значение для наклона отрицательной части, что может потребовать дополнительного подбора и настройки.

Функцию активации LeakyReLU применяют в нейронных сетях в основном в скрытых слоях для предотвращения проблемы "умерших" нейронов, особенно в глубоких архитектурах.

##  GELU (Gaussian Error Linear Unit)

GELU — функция активации, которая сочетает в себе свойства ReLU и сигмоиды. Она применяет к входным данным кумулятивную функцию гауссова распределения.

На практике GELU может быть приблизительно вычислена так:

$$\large \text{GELU}(x)\approx 0.5x(1+\tanh[\sqrt{2/\pi}(x+0.044715x^3)])$$

или

$$\large \text{GELU}(x) \approx x\cdot \sigma(1.702x)$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/gelu_function.png" width="500"></center>

Значения GELU лежат в диапазоне от $0$ до $+\infty$. Она не требует дополнительной настройки гиперпараметров.

GELU применяется в современных нейронных сетях, таких как трансформеры (о которых поговорим в следующих лекциях), и широко используется в архитектурах, требующих адаптивности и высокой производительности.

[[doc] 🛠️ GELU в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html):
```python
torch.nn.GELU()
```

In [ ]:
activation = nn.GELU()
input_values = torch.randn(5) * 5
activation_gelu = activation(input_values)
print(f"input_values: {input_values}\nactivation_gelu: {activation_gelu}")

#  Функции потерь (loss functions)

Предположим, у нас есть нейронная сеть с некоторыми весами. Прежде всего мы должны понять, насколько она точна, то есть насколько наши ожидания соответствуют результату работы нейронной сети. Мы подали на вход нейронной сети пример данных, сигналы прошли через наши слои и функции активации вперёд **(forward propagation)**, и на выходе мы имеем предсказание. Как оценить его точность?

Для оценки соответствия полученного результата ожидаемому используют функцию потерь (loss function). Значение функции потерь даёт количественную оценку величины такого соответствия.

Функция потерь в нейронной сети принимает два аргумента:
* вектор значений, который мы считаем априорно верным;
* вектор значений конечных выходов модели, который должен соответствовать априорно верному.

[[doc] 🛠️ Функции потерь в PyTorch](https://pytorch.org/docs/stable/nn.html#loss-functions)

###  Cross-Entropy

Кросс-энтропия — классическая функция потерь **при решении задач классификации**. Возникновение такой формы измерения различия между целевой переменной и предсказанием модели в задаче классификации мы уже рассматривали на лекции про линейные классификаторы. Напомним здесь формулу, по которой она рассчитывается.

Для $i$-го объекта выборки, если выходной вектор состоит из $C$ компонент (**логитов** для $C$ классов), кросс-энтропия между выходом модели $\hat{y}$ и целевым вектором $y$ будет равна

$$\large \text{CE}_i(\hat{y},y)= - \sum_{k=1}^{C}{y_{ik}\cdot\log\left(\frac{\exp(\hat{y}_{ik})}{\sum_{j=1}^{C}\exp(\hat{y}_{ij})}\right)}$$


**Примечание, относящееся ко всем функциям потерь в PyTorch, которые будут рассмотрены ниже.**

При вычислении по всему набору данных (или по мини-батчу) из $N$ объектов ошибка на отдельных объектах может усредняться или суммироваться. За это отвечает параметр `reduction`, который принимает значения `'mean'` или `'sum'`, а также может принимать значение `'none'`, при котором агрегация производиться не будет, и тогда функция будет возвращать тензор значений размером $N$.

**По умолчанию в большинстве функций потерь в PyTorch `reduction='mean'`.**


[[doc] 🛠️ Cross-Entropy Loss в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss):
```python
torch.nn.CrossEntropyLoss()
```

Обратите внимание, что Cross-Entropy Loss в PyTorch уже включает в себя Softmax и принимает в качестве выхода модели логиты. Поэтому при использовании данной фукции потерь ставить на последнем слое нейронной сети Softmax **не нужно**.

In [ ]:
import torch
from torch import nn

criterion = nn.CrossEntropyLoss()


# fmt: off
model_output = torch.tensor([[2.4, 1.9, 7.3],
                             [9.5, 2.7, 4.0],
                             [5.7, 4.1, 0.2]])  # logits
# fmt: on

print(f"model_output:\n {model_output}")

target = torch.tensor([2, 0, 1], dtype=torch.long)  # class labels
print(f"target: {target}")

loss_ce = criterion(model_output, target)
print(f"loss_ce: {loss_ce}")

$$\text{CE}_1 = - \log\left(\frac{\exp{(7.3)}}{\exp{(2.4)}+\exp{(1.9)}+\exp{(7.3)}}\right)$$

$$\text{CE}_2 = - \log\left(\frac{\exp{(9.5)}}{\exp{(9.5)}+\exp{(2.7)}+\exp{(4.0)}}\right)$$

$$\text{CE}_3 = - \log\left(\frac{\exp{(4.1)}}{\exp{(5.7)}+\exp{(4.1)}+\exp{(0.2)}}\right)$$

$$\text{CE} = \frac{1}{3}(\text{CE}_1 + \text{CE}_2 + \text{CE}_3)$$

In [ ]:
import numpy as np

ce_1 = -np.log(np.exp(7.3) / (np.exp(2.4) + np.exp(1.9) + np.exp(7.3)))
ce_2 = -np.log(np.exp(9.5) / (np.exp(9.5) + np.exp(2.7) + np.exp(4.0)))
ce_3 = -np.log(np.exp(4.1) / (np.exp(5.7) + np.exp(4.1) + np.exp(0.2)))

ce = (1 / 3) * (ce_1 + ce_2 + ce_3)
print(f"hand-calculated loss_ce: {ce}")

* **Преимущества:** Важное свойство кросс-энтропии — возможность работать с весами для классов, а значит и возможность применения этой функции потерь при работе с несбалансированным датасетом.

#### Веса классов

При работе с несбалансированными данными модель может сосредоточиться на доминирующем классе, игнорируя признаки объектов. Например, в датасете с 95% объектов класса 1 и 5% — класса 0, модель может просто присваивать всем объектам класс 1, достигая 95% точности.

Чтобы учесть дисбаланс, можно модифицировать функцию потерь, увеличивая штраф за ошибки в минорных классах. В PyTorch параметр `weight` в `CrossEntropyLoss` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) позволяет задать веса для каждого класса, корректируя функцию потерь с учетом дисбаланса.

Посмотрим, как это работает. Допустим, мы получили от нейросети неверные предсказания: второй объект должен относиться к классу $1$, а не $0$:

In [ ]:
# fmt: off
# Scores for batch of two samples
model_output = torch.tensor([[30.0, 2.0],
                             [30.0, 2.0]])

target = torch.tensor([0, 1])  # Second sample belongs to class 1
# but logit for class 0 is greater: 30 > 2. So it was misclassified
# fmt: on

Подсчитаем Cross-Entropy Loss без весов:

$$\large \text{CE} = \frac{1}{2} \biggr[- \log\frac{e^{30}}{e^{30}+e^{2}} - \log\frac{e^{2}}{e^{30}+e^{2}}\biggr]\approx 14.0 $$

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(model_output, target)
print(f"Loss = {loss.item():.2f}")

Если у нас есть два класса с соотношением $4:1$, можно задать веса `weight = [0.2, 0.8]`. И, так как сеть ошиблась на классе с большим весом, ошибка вырастет:

$$\large \text{CE}_{\text{weighted}} =  \biggr[\mathbf{-0.2} \log\frac{e^{30}}{e^{30}+e^{2}} -\mathbf{0.8} \log\frac{e^{2}}{e^{30}+e^{2}}\biggr]\approx 22.4 $$

In [ ]:
weights = torch.tensor([0.2, 0.8])
criterion = torch.nn.CrossEntropyLoss(weight=weights)
loss = criterion(model_output, target)
print(f"Loss = {loss.item():.2f}")

Сумма весов не обязана быть равной единице, важно только соотношение между весами. Часто соотношение весов делают обратно пропорциональным количеству объектов в классах.

In [ ]:
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([1.0, 4.0]))
loss = criterion(model_output, target)
print(f"Loss = {loss.item():.2f}")

- Иногда качество модели можно улучшить, взяв квадратные корни от полученных таким образом весов (немного снижает штрафы за ошибки на редких классах).

- Несмотря на интуитивно понятную логику работы способа, он не всегда дает значительный эффект. Тем не менее, на практике стоит пробовать экспериментировать с этим способом наряду с прочими техниками борьбы с дисбалансом.

### Negative Log Likelihood


NLL Loss, используемая для классификации, отличается от Cross-Entropy Loss тем, что ожидает логарифмы вероятностей классов на выходе модели, а не логиты.

Для $i$-го объекта выборки, если выходной вектор состоит из $C$ компонент (**логарифмов вероятностей** для $C$ классов), обратный логарифм правдоподобия между выходом модели $\hat{y}$ и целевым вектором $y$ будет равен:

$$\large \text{NLL}_i(\hat{y},y)= - \sum_{k=1}^{C}{y_{ik}\cdot\hat{y}_{ik}}$$


Для применения NLL Loss в классификации логиты модели нужно сначала преобразовать через Softmax и взять их натуральный логарифм. В PyTorch это выполняется с помощью модуля `LogSoftmax` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html). Связь между NLL Loss и Cross-Entropy Loss можно выразить так:

<center><img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/ce_loss_vs_nll_loss.png" width="900"></center>

Двойственность NLL Loss и Cross-Entropy Loss может немного путать. Она возникла из-за того, что NLL Loss была исторически раньше реализована в библиотеке. Было принято на выходе модели ставить LogSoftmax, и использовать NLL Loss. Позднее была реализована функция Cross-Entropy Loss, которая включала в себя одновременно LogSoftmax и NLL Loss и позволяла не добавлять в модель лишний модуль. За счет такого упрощения использование Cross-Entropy Loss быстро стало более популярно. Реализацию NLL Loss, по всей видимости, оставили в библиотеке скорее для обратной совместимости.

Исторически NLL Loss использовалась с LogSoftmax, тогда как Cross-Entropy Loss объединяет LogSoftmax и NLL Loss, упрощая модель. Cross-Entropy Loss стала популярнее за счёт упрощения.

После обучения модели:

- **LogSoftmax + NLL Loss:** На выходе логарифмы вероятностей. Для получения вероятностей нужно взять экспоненту.
  
- **Cross-Entropy Loss:** На выходе логиты. Для получения вероятностей примените Softmax.

[[blog] ✏️ Объяснение Negative Log Likelihood Loss](https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/)

[[blog] ✏️ О соотношении Cross-Entropy Loss и Negative Log Likelihood Loss](https://jamesmccaffrey.wordpress.com/2020/06/11/pytorch-crossentropyloss-vs-nllloss-cross-entropy-loss-vs-negative-log-likelihood-loss/)

[[blog] ✏️ Объяснение Negative Log Likelihood Loss](https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/)

[[blog] ✏️ О соотношении Cross-Entropy Loss и Negative Log Likelihood Loss](https://jamesmccaffrey.wordpress.com/2020/06/11/pytorch-crossentropyloss-vs-nllloss-cross-entropy-loss-vs-negative-log-likelihood-loss/)

###  Binary Cross-Entropy

В частном случае, когда количество классов равно двум (**задача бинарной классификации**), их можно закодировать одним числом: $0$ — для первого класса, и $1$ — для второго. Тогда сумму $\displaystyle \sum_{k=1}^{C}$ в формуле Cross-Entropy Loss можно расписать в явном виде.

Для $i$-го объекта выборки, когда выход модели является скаляром (**вероятностью** отнесения объекта к классу $1$), бинарная кросс-энтропия между выходом модели $\hat{y}$  и целевым значением $y$ будет равна

$$\text{BCE}_i(\hat{y},y)= - [{y_i\cdot\log(\hat{y_i})+(1-y_i)\cdot\log(1-\hat{y_i})}]$$

[[doc] 🛠️ Binary Cross-Entropy Loss в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html#torch.nn.BCELoss):

```python
torch.nn.BCELoss()
```



Важной особенностью BCE Loss является то, что здесь используется не one-hot кодирование целевых значений для двух классов, а **одно число: 0 — первый класс, 1 — второй класс.** При этом значения целевой переменной должны быть представлены как вещественные (float) числа.

In [ ]:
criterion = nn.BCELoss()

model_output = torch.rand(1)
print(f"model_output: {model_output}")

target = torch.empty(1).random_(2)
print(f"target: {target}")

loss_bce = criterion(model_output, target)
print(f"loss_bce: {loss_bce}")

### Binary Cross-Entropy With Logits

Для того чтобы выход модели при бинарной классификации представлял собой **вероятность отнесения объекта к классу $1$**, на выходе модели мы должны использовать логистическую функцию (sigmoid), результат которой можно будет передать в BCE Loss.

По аналогии с NLL Loss и Cross-Entropy Loss, у BCE Loss есть своя "пара": BCE With Logits Loss. Эта функция потерь совмещает в себе две операции:

* применение логистической функции Sigmoid,
* расчет BCE Loss.

Как можно понять из названия, BCE With Logits Loss на вход ожидает логиты. Взаимоотношение между BCE Loss и BCE With Logits Loss можно отобразить такой иллюстрацией:

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/bce_loss_vs_bce_with_logits_loss.png" width="900"></center>

Функцию потерь BCE Loss (или ее "аналог" BCE With Logits Loss) можно применять не только в случае бинарной классификации, но и **в случае Multi-label классификации**.

В таком случае в векторе целевых значений вероятностей для каждого объекта может быть больше, чем одна единица. А логиты на последнем слое сети независимо пропускаются через сигмоиду (а не через Softmax, как в случае с многоклассовой классификацией).

#  Метод обратного распространения ошибки

Для эффективного численного расчета градиента функции потерь по обучаемым параметрам модели применяется **метод обратного распространения ошибки (backpropagation)**. Благодаря данному методу становится практически возможным использование метода градиентного спуска для проведения процедуры обучения.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/backward_pass.png" width="450"></center>

Метод обратного распространения ошибки использует структуру многослойной нейронной сети как сложной функции, применяя правило дифференцирования сложной функции для вычисления градиента от функции потерь по весам сети. Градиент от функции потерь вычисляется при движении по нейронной сети от её выходов в направлении входов. Именно такой порядок обхода вычислительного графа и обуславливает название метода.

## Вычислительный граф

По существу, нейронная сеть является сложной функцией, работу которой можно представить как последовательное выполнение математических операций. Такое представление функций называется [вычислительным графом ✏️[blog]](https://qudata.com/ml/ru/ML_Comp_Graph.html).

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/nn_fully_connected.png"  width="500"></center>

Алгоритм обратного распространения ошибки позволяет находить градиенты для любого графа вычислений, если описываемая им функция дифференцируема.

В его основе лежит правило взятия производной сложной функции (chain rule):

Одна переменная:

$$\large y(x) = f(u(g(x))) $$

$$\large \frac{dy}{dx} = \frac{df}{du} \frac{du}{dg} \frac{dg}{dx}$$

Несколько переменных:

$$\large y(x) = f(u_1(x),u_2(x),...u_n(x))$$

$$\large \frac{dy}{dx} = \sum_{i=1}^{n} \frac{\partial f(u_1, u_2, ... u_n)}{\partial u_i} \frac{du_i}{dx}$$

$$\large \underbrace{\frac{d}{dx} f(\vec{\mathbf{u}}(x))}_{\text{Derivative of composition function}} = \overbrace{\nabla_{\vec{u}} f \cdot \vec{\mathbf{u}}'(x)}^{\text{Dot product of vectors}}$$

##  Пошаговый разбор метода обратного распространения

Пусть в каком-то узле графа производится вычисление

$$\large z = f(x, y),$$

и далее результат вычисления $\large z$ используется для вычисления функции $\large L(z)=L(f(x, y))$.

Тогда правило вычисления производных $\dfrac{\partial L}{\partial x}$ и $\dfrac{\partial L}{\partial y}$ можно представить следующим образом:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/rule_for_taking_gradients.png"  width="500"></center>

Рассмотрим следующую функцию:

$$\Large f(w,x)=\frac{1}{1+e^{-(w_0x_0+w_1x_1+w_2)}}$$

Представим ее в виде вычислительного графа, состоящего из элементарных операций, от которых просто берутся производные:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/graph_of_calculation_gradient.png"  width="700"></center>

На примере данной функции рассмотрим алгоритм обратного распространения ошибки и найдём величину её градиента по параметрам $\large w$.
Нам потребуется вычислить частные производные $\dfrac{\partial f}{\partial w_0}, \dfrac{\partial f}{\partial w_1}, \dfrac{\partial f}{dw_2}, \dfrac{\partial f}{\partial x_0}$ и $\dfrac{\partial f}{\partial x_1}$.

Пусть "веса" $w$ инициализированы значениями $w_0=2,\;w_1=-3,\;w_2=-3$, а "входные признаки" $x$ принимают значения $x_0=-1,\;x_1=-2$.

Делая прямой проход через граф вычислений для данной функции, получаем её значение для заданных $w$ и $x$ равным $f=0.73$:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/forward_pass_example.png" width="800"></center>

Далее, в соответствии с алгоритмом обратного распространения ошибки, рассчитаем частные производные, пройдясь последовательно по графу вычислений, постепенно накапливая искомое значение для градиента функции.

Для начала зададим $\dfrac{df}{df}=1$.

Начинаем обратный проход по графу вычислений — первая вершина содержит функцию $f(x)=\dfrac{1}{x}$, производная которой равна $\dfrac{df}{dx}=-\dfrac{1}{x^2}$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/compute_gradient_1_step.png" width="800"></center>

$$\large f(x)=\frac1x \quad \longrightarrow \quad \frac{df}{dx} = -\frac{1}{x^2}$$

В следующем узле находится функция $f(x)=1+x$. Производная от выражения в данном узле равняется $\dfrac{df}{dx}=1$:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/compute_gradient_2_step.png" width="800"></center>

$$\large f(x)=c+x \quad \longrightarrow \quad \frac{df}{dx} = 1$$

Третья вершина содержит экспоненту $f(x)=e^x$. Её производная также является экспонентой $\dfrac{df}{dx}=e^x$:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/compute_gradient_3_step.png" width="800"></center>

$$\large f(x)=e^x \quad \longrightarrow \quad \frac{df}{dx} = e^x$$

Следующая вершина, четвертая, содержит умножение на константу $f(x)=ax$. Производная равна $\dfrac{df}{dx}=a$ (в данном случае $a=-1$):

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/compute_gradient_4_step.png" width="800"></center>

$$\large f(x)=ax \quad \longrightarrow \quad \frac{df}{dx} = a$$

Двигаясь по графу вычислений, мы дошли до узла суммирования, который имеет два входа. Относительно каждого из входов локальный градиент в вершине суммирования будет равен $1$:
$$\large f(x,y)=x+y \quad \Rightarrow \quad \frac{\partial f}{\partial x}=1  \quad \quad \frac{\partial f}{\partial y}=1$$
Так как умножение на единицу не изменит значения входного градиента, всем входам узла суммирования мы можем приписать точно такое же значение входного градиента ($0.2$), что мы имели и для самого узла суммирования. Будем действовать аналогично и со всеми остальными узлами суммирования, которые встретятся нам в вычислительном графе.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/compute_gradient_5_step.png" width="800"></center>

Двигаясь далее к началу графа вычислений, мы подходим к вершинам умножения. Для такой вершины локальный градиент по отношению к какому-либо из входов будет равен значению оставшегося входа. Остается умножить локальный градиент на входящий.

$$\large f(w,x)=wx \quad \Rightarrow \quad \frac{\partial f}{\partial w}=x  \quad \quad \frac{\partial f}{\partial x}=w$$

Точно так же мы можем поступить и с оставшейся второй вершиной умножения, которая привязана к $w_1$ и $x_1$:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/compute_gradient_6_step.png" width="800"></center>

Так, двигаясь по графу вычислений в обратном направлении от выхода функции к входным аргументам, мы последовательно для каждого узла умножаем локальный градиент на входящий градиент, используя цепное правило дифференцирования сложной функции. В описанном примере мы полностью разбили граф вычислений на отдельные элементарные узлы. Разбиение вычислительного графа на элементарные узлы вовсе не обязательно — мы можем сгруппировать несколько вершин вместе, если они образуют дифференцируемую функцию, от которой "удобно" брать производную, и рассматривать их совместно.

В нашем примере мы можем заметить, что вычислительный граф можно свести к двум операциям: получению выражения $w_0x_0+w_1x_1+w_2$ и последующему вычислению от него сигмоидальной функции.

Функция сигмоиды:

$$\large \displaystyle \sigma(x) = \frac{1}{1+e^{-x}}.$$

Важно отметить, что сигмоида обладает важным свойством: её производная может быть выражена через саму сигмоидальную функцию:

$$\large \frac{d}{dx}\sigma(x) = \frac{d}{dx}(1+e^{-x})^{-1} = \frac{e^{-x}}{(1+e^{-x})^{2}} = \frac{1}{1+e^{-x}} \cdot \frac{1+e^{-x}-1}{1+e^{-x}} = \sigma(x)\cdot(1-\sigma(x))$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/compute_gradient_join_vertices_sigmoid_example.png" width="800"></center>

В коде без использования библиотек подсчёт градиентов можно записать как:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/calculating_gradients_in_code.png" width="800"></center>

Для базовых операций можно выделить следующие шаблоны "разделения" градиента:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/add_copy_mul_max_gates.png" width="700"></center>

##  Анимация работы метода обратного распространения ошибки



Таким образом, метод обратного распространения ошибки включает в себя следующие шаги:
* Forward pass (FP) — прямое распространение сигнала от входа к выходам (без которого не получить вычисленные значения в графе).
* Backward pass (BP) — расчёт величины градиента функции потерь по весам от выходного слоя ко входному.
* Обновление весов в зависимости от величины градиента. На анимации буквой $\eta$ обозначен learning rate.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/backprop_animation.gif" width="600"></center>

<center><em>Источник: <a href="https://robocraft.ru/algorithm/560">Принцип обучения многослойной нейронной сети с помощью алгоритма обратного распространения</a></em></center>

Для более детального рассмотрения принципов работы и обучения нейронных сетей рекомендуем:

[[video] 📺 Плей-лист от 3Blue1Brown](https://youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&si=O3gq6RoIxoNS6iJf) [[озвучка на русском](https://youtube.com/playlist?list=PLZjXXN70PH5itkSPe6LTS-yPyl5soOovc&si=ElKVi98Ui4m2nJKq)].

## Автоматическое вычисление градиента в PyTorch

PyTorch умеет запоминать последовательность операций с тензорами и вычислять градиент.

In [ ]:
import torch

x = torch.tensor([-1.0, -2.0])  # x from above example

x = torch.cat([x, torch.tensor([1.0])])  # concatenate x with 1. for bias trick

W = torch.tensor([2.0, -3.0, -3.0], requires_grad=True)  # w from above example

print(f"W.grad = {W.grad} (before forward and backward pass grad is 'None')")

# forward pass to compute f
s = x.matmul(W)
f = torch.sigmoid(s)
print(f"f(x, W) = {f:.2f}")

# backward pass to compute gradient df/dW
f.backward()
print(f"W.grad = {W.grad}")

Получили такие же значения частных производных $\dfrac{\partial f}{\partial w_i}$, как и при вычислении вручную (на иллюстрации градиент расчитан с точностью до двух знаков после запятой, в коде выше получены более точные значения).

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/autograd_example.png" width="700"></center>

Мы не требовали возможности расчета градиента по аргументу `x` (не указали `requires_grad=True`), поэтому после вызова `f.backward()` градиент по нему не рассчитается:

In [ ]:
print(f"x.grad = {x.grad}")

Отсоединение тензора от графа вычислений производится с помощью `.detach()` (используется, если нужно производить какие-либо дальнейшие операции с тензором, но чтобы они не становились частью вычислительного графа):

In [ ]:
f_detached = f.detach()

print(f"f_detached = {f_detached:.2f}")
print(f"f_detached type: {type(f_detached)}")

Также от тензора, содержащего скаляр, можно получить его величину с помощью `.item()`:

In [ ]:
value = f_detached.item()

print(f"value = {value:.2f}")
print(f"value type: {type(value)}")

# Представление текстовых данных

Рассмотренные нами этапы построения и обучения нейронной сети применимы к решению любой задачи. Однако данные не всегда поступают в том формате, который требуется для работы модели.

Какую бы мы задачу ни решали, в нейронную сеть нужно подавать числа. Поскольку мы работаем с текстами, необходимо предварительно векторизовать их. Давайте посмотрим, как мы можем представить текстовую информацию.

## Слой эмбеддингов

Ранее мы применяли "мешок слов" и алгоритм TF-IDF для представления наших слов. Проблемы возникают, когда пространство объектов начинает расти и у нас возникают огромные разреженные матрицы.

Кроме того, некоторые объекты у нас сразу могут быть ближе: семантически "король" и "королева" отличаются только полом, различие между словами "король" и "стул" заметно выше.

Поэтому мы можем переводить наши слова в вектора меньшей размерности, которые при этом будут сравнимы между собой с помощью модуля `nn.Embedding` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html).

[[blog] ✏️ Lena Voita NLP Course](https://lena-voita.github.io/nlp_course/word_embeddings.html)

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/word_representation_intro_min.png" width="600"></center>

____

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/lookup_table.gif" width="600"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/word_embeddings.html">Lena Voita NLP Course</a></em></center>

In [ ]:
# Let's say you have 2 sentences (lowercased, punctuations removed):
sentences = "i am new to pytorch i am having fun"

words = sentences.split(" ")

print(f"All words: {words} \n")

vocab = set(words)  # create a vocabulary
vocab_size = len(vocab)

print(f"Vocabulary (unique words): {vocab} \n")
print(f"Vocabulary size: {vocab_size} \n")

# map words to unique indices
word2idx = {word: ind for ind, word in enumerate(vocab)}

print(f"Word-to-id dictionary: {word2idx} \n")

encoded_sentences = [word2idx[word] for word in words]

print(f"Encoded sentences: {encoded_sentences}")

Теперь нейросетевой слой эмбеддингов может быть определён так:

In [ ]:
import torch.nn as nn

# let's say you want embedding dimension to be 3
emb_dim = 3

emb_layer = nn.Embedding(vocab_size, emb_dim)
word_vectors = emb_layer(torch.LongTensor(encoded_sentences))

print(f"Shape of encoded sentences: {word_vectors.shape} \n")
print(f"Shape of weigths: {emb_layer.weight.shape}")

Этот код инициализирует эмбеддинги согласно нормальному распределению (со средним значением 0 и дисперсией 1). Таким образом, пока что никакого различия или сходства между векторами нет.

`word_vectors` — тензор размером (9, 3). 9 слов в датасете, размер 3 задан нами.

`emb_layer` имеет 1 обучаемый параметр `weight`, который по умолчанию True. Можем проверить так:

In [ ]:
emb_layer.weight.requires_grad

Существуют алгоритмы для получения предобученных векторных представлений, которые потом могут использоваться в других задачах. Мы рассмотрим их в следующем разделе.

## Алгоритм Word2Vec

**Word2vec** — группа алгоритмов для получения векторных представлений слов в многомерном пространстве.

**Векторные представления слов назваются эмбеддингами.**

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/w2v_example.png" width="350"></center>

<center><em>Источник: <a href="https://habr.com/ru/articles/585838/">Семантика и технология Word2Vec</a></em></center>



Word2Vec реализуется при помощи полносвязной нейронной сети из двух слоев, которая обрабатывает текст, преобразуя его в числовые "векторизованные" слова. Основной особенностью w2v стало добавление контекста: нейросеть использует информацию о том, какие слова находятся рядом в заданном окне.

Входные данные w2v — это громадный текстовый корпус, из которого на выходе мы получаем пространство векторов, размерность которого обычно достигает сотен, где каждое уникальное слово в корпусе представлено вектором из сгенерированного пространства.

**Данные**

Исходные, "сырые" данные представляют собой предложение

**The quick brown fox jumps over the lazy dog**.

Оставляя за скобками приведение к каноничным формам, создаем датасет так:
1. Проходим по тексту окном заданного размера (в данном случае размер окна 2 означает 2 слова слева и два слова справа от текущего).
2. Относительно текущей позиции формируем пары "текущее слово-соседнее слово" (биграммы).
3. Смещаем окно вправо.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/vectorization_example.jpeg" width="700"></center>

<center><em>Источник: <a href="https://habr.com/ru/articles/778048/">Краткий обзор техник векторизации в NLP</a></em></center>

Есть два основных способа реализации Word2Vec: **Skip-Gram** и **CBOW**.

* **Слева** расположен метод **Skip-Gram**, в котором мы передаем какое-то слово нашей нейронной сети и просим ее предположить возможный контекст. $w[i]$ — слово, которое передается на вход и которое находится на позиции $i$ в предложении. На выходе имеем два предшествующих слова и два последующих слова с учетом позиции $i$.

* **Справа — CBOW** (Continuous Bag of Words). Здесь, вместо того, чтобы предугадывать контекстные слова, добавляем их в модель и просим модель предсказать текущее слово.

Фокус состоит в том, что на самом деле никакой обученной нейросети у нас нет. Здесь другая цель — изучить веса скрытого слоя по мере того, как корректируются спрогнозированные веса окружающих слов. Эти веса и есть эмбеддинги.

<div align="center">
<table>
<tr>
<td>

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/skip_gram.jpeg" width="400"></center>
    
</td>
<td>

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/cbow.jpeg" width="400"></center>

</td>  

<tr>
<td>



<center><em>Источник: <a href="https://habr.com/ru/articles/778048/">Skip-Gram</a></em></center>
    
</td>
<td>

<center><em>Источник: <a href="https://habr.com/ru/articles/778048/">CBOW</a></em></center>


</td>  

</table>
</div>

Рассмотрим, как учится сеть для варианта Skip-Gram. На вход сети приходит One-Hot вектор первого слова из биграммы, преобразовывается в пространство эмбеддингов в скрытом слое и поступает на слой SoftMax (размером с весь словарь, т.е. размер входа равен размеру выхода).

По факту нейросеть должна выучить вероятность существования слова *quick* при приходе на вход слова *fox*. При этом можно отметить, что слову *fox* также соответствуют слова *brown, jumps, over*. Обучаясь по кросс-энтропии, последний слой должен выдавать примерно одинаковые вероятности всех этих слов.


<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/skip_gram_example.jpeg" width="700"></center>

<center><em>Источник: <a href="https://habr.com/ru/articles/778048/">Краткий обзор техник векторизации в NLP</a></em></center>

В результате обучения на разнообразных данных сеть выучивает такие распределения, что вектора, соответствующие похожим словам, располагаются близко в новом пространстве, и над ними работают векторные операции.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/w2v_example_2.png" width="500"></center>

<center><em>Источник: <a href="https://habr.com/ru/articles/446530/">Word2vec в картинках</a></em></center>


Разумеется, эти операции не приводят к точному соответствию, однако получаемые вектора оказываются самыми похожими.

Обучение CBOW несколько отличается. На вход сети подаётся $C$ контекстных слов в виде One-Hot, после перемножения с линейным слоем результат усредняется. На выходе слой SoftMax такого же размера.

[[colab] 🥨 Подробное руководство по использованию Word2Vec](https://colab.research.google.com/drive/1OJF0k-E60sp9Vyoj1yWuoedGTrwRiTyj?usp=sharing)

### Тестирование векторных представлений

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки [Gensim 🛠️[doc]](https://radimrehurek.com/gensim/auto_examples/index.html#documentation).

In [ ]:
!pip install gensim -q

In [ ]:
import gensim

Загрузим предобученную модель от Google. Модель включает в себя векторы для 3 миллионов слов и обучалась на 100 миллиардах слов из набора данных Google News. Длина вектора равна 300.

Для загрузки предобученных векторных представлений используется модуль `KeyedVectors`.

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/datasets/GoogleNews-vectors-negative300.bin.gz

In [ ]:
from gensim.models import KeyedVectors
google_news_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
print(f"Size of vector 'coffee': {google_news_model['coffee'].shape}")
print(f"Size of vector 'tea': {google_news_model['tea'].shape}")
print(f"Size of vector 'ball': {google_news_model['ball'].shape}")
print(f"Size of vector 'crocodile': {google_news_model['crocodile'].shape}")

Протестируем загруженные векторные представления.

#### Косинусная мера

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/cosine_similarity.png" width="550"></center>

<center><em>Источник: <a href="https://predictivehacks.com/a-high-level-introduction-to-word-embeddings/">A High-Level Introduction to Word Embeddings</a></em></center>


In [ ]:
print(f"Cosine similarity between 'coffee' and 'tea': \
{round(google_news_model.similarity('coffee','tea').item(), 2)}")

In [ ]:
print(f"Cosine similarity between 'ball' and 'crocodile': \
{round(google_news_model.similarity('ball','crocodile').item(), 2)}")

#### Близкие слова

In [ ]:
print(f"Top-5 most similar words to 'cat':")
for x in google_news_model.most_similar('cat', topn=5):
    print(f"{x[0]} {round(x[1], 2)}")

In [ ]:
print(f"Top-5 most similar words to 'bad':")
for x in google_news_model.most_similar('bad', topn=5):
    print(f"{x[0]} {round(x[1], 2)}")

In [ ]:
print(f"Top-5 most similar words to 'good':")
for x in google_news_model.most_similar('good', topn=5):
    print(f"{x[0]} {round(x[1], 2)}")

#### Семантические аналогии

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/semantic_proportions.png" width="1000"></center>

<center><em>Источник: <a href="https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/">Understanding Word Embeddings: From Word2Vec to Count Vectors</a></em></center>


king - man + woman = ?

In [ ]:
print(f"king - man + woman = \
{google_news_model.most_similar(positive=['king','woman'], negative=['man'], topn=1)[0][0]}")

Для глагольных форм: swimming - walking + walked = ?

In [ ]:
print(f"swimming - walking + walked = \
{google_news_model.most_similar(positive=['swimming','walked'], negative=['walking'], topn=1)[0][0]}")

Для стран и городов: Turkey - Russia + Moscow = ?

In [ ]:
print(f"Turkey - Russia + Moscow = \
{google_news_model.most_similar(positive=['Turkey','Moscow'], negative=['Russia'], topn=1)[0][0]}")

#### Поиск лишнего слова

- вычисляется среднее значение всех слов-векторов (центр);
- вычисляется косинусное расстояние от центра до каждого слова;
- возвращается единственное слово с наибольшим косинусным расстоянием от центра.

In [ ]:
words = ['spoon', 'fork', 'knife', 'plate']
print(f"The word '{google_news_model.doesnt_match(words)}' \
from the given list {words} doesn’t go with the others.")

In [ ]:
words = ['sorrow', 'delight', 'happiness', 'joy']
print(f"The word '{google_news_model.doesnt_match(words)}' \
from the given list {words} doesn’t go with the others.")

### Использование предобученных эмбеддингов

Если мы хотим использовать заранее обученные векторные представления, мы можем заморозить веса слоя эмбеддингов.

In [ ]:
emb_layer.weight.requires_grad = False

У модуля `nn.Embedding` есть метод `from_pretrained`, в который можно передать предобученную матрицу эмбеддингов. При этом по умолчанию они не будут обучаться — за это отвечает флаг `freeze`.

In [ ]:
print(f"Vectors shape in Google news model is {google_news_model.vectors.shape}")

weights = torch.FloatTensor(google_news_model.vectors)
google_news_emb = nn.Embedding.from_pretrained(weights, freeze=True)

In [ ]:
input = torch.LongTensor(encoded_sentences)
print(f"Shape of data after embedding layer: {google_news_emb(input).shape}")

# Углубление в PyTorch. Пример нейронной сети

## Работа с данными

Предварительная обработка данных часто сильно зависит от домена, предметной области, самих данных. В идеале мы бы хотели, чтобы код, относящийся к набору данных, был отделен от кода для обучения модели для его лучшей читаемости, понимаемости и модульности.

PyTorch предоставляет два базовых класса для работы с данными: `torch.utils.data.DataLoader` и `torch.utils.data.Dataset`, которые позволяют работать как со встроенными наборами данных, так и с вашими собственными данными.

`Dataset` хранит в себе объекты (samples, сэмплы) — например, тексты и соответствующие им метки (labels, targets).

`DataLoader` представляет из себя итерируемый объект — обертку над `Dataset`-ом, и позволяет получить простой доступ к объектам и меткам из набора данных в виде мини-батчей.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/dataset_dataloader_nlp.png" width="1000"></center>

### Загрузка и предобработка данных

В качестве данных будем использовать русскоязычный корпус коротких текстов из статьи:

[[paper] 🎓 Автоматическое построение и анализ корпуса коротких
текстов (постов микроблогов) для задачи разработки и
тренировки тонового классификатора (Ю. Рубцова, 2012)](https://study.mokoron.com/wp-content/uploads/2014/02/rubtsova-kesw_paper-21.pdf)


Сообщения разбиты на два класса: с положительной окраской и с отрицательной окраской.

Загрузим данные уже после предобработки.

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/datasets/RuTweetCorp.csv

In [ ]:
import pandas as pd
df = pd.read_csv('RuTweetCorp.csv')
print(f"Dataframe shape: {df.shape}\n")
print(f"Number of texts in class: {df['tone'].value_counts()}\n")
df.head()

Для удобства преобразуем тексты из строк в списки токенов.

In [ ]:
df['preprocessed'] = df['preprocessed'].str.split()
df.head()

Соберем все уникальные слова в словарь `vocabulary` и посчитаем количество вхождений каждого слова.

Создадим словарь с индексами `word2id`.
- Добавим в качестве ключа спецсимвол паддинга и зададим ему дефолтный индекс 0.
- Выделим слова, встретившиеся больше 10 раз.
- Далее каждому слову (ключу) присвоим его порядковый номер (значение).

Это позволит перевести предложение в численный вид (вектор).

Также cоздадим обратный словарь `id2word`. В нем наоборот ключами являются индексы, а значениями — слова. Этот словарь позволяет раскодировать предложение, чтобы оценить предсказание модели.

In [ ]:
from collections import Counter

vocabulary = Counter()
for text in df['preprocessed']:
    vocabulary.update(text)
print(f'Total unique tokens: {len(vocabulary)}\n')

word2id = {'PAD':0}
for word in vocabulary:
    if vocabulary[word] > 10:
        word2id[word] = len(word2id)
print(f'Mapping word2id dictionary: {word2id}\n')

id2word = {i:word for word,i in word2id.items()}
print(f'Mapping id2word dictionary: {id2word}\n')

print(f'Unique tokens that appeared more than 10 times: {len(word2id)}')

### Паддинг

Предложения в обучающей выборке имеют разную длину. Каждое предложение представляется в виде вектора. Они объединяются в матрицу и образуют батч. Но в матрице все строки должны иметь одинаковую длину. Поэтому необходимо уравнять длину предложения. Для этого используются паддинги. Мы добиваем паддингами (паддим) предложения до некоторой длины, то есть добавляем нужное количество паддингов в конец предложения.

В дальнейшем мы будем использовать метод `torch.nn.utils.rnn.pad_sequence`. Он позволяет добивать паддингами все последовательности в батче до максимальной длины в батче.

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

X = []
for tokens in df.preprocessed[:3]:
    ids = torch.LongTensor([word2id[token] for token in tokens if token in word2id])
    X.append(ids)
X = pad_sequence(X, batch_first=True)

print("Sentence 1 in batch")
print(f"Padded sequence of indexes: {X[0]}")
print(f"Padded sequense of tokens: {[id2word[int(_)] for _ in X[0]]}\n")
print("Sentence 2 in batch")
print(f"Padded sequence of indexes: {X[1]}")
print(f"Padded sequense of tokens: {[id2word[int(_)] for _ in X[1]]}\n")
print("Sentence 3 in batch")
print(f"Padded sequence of indexes: {X[2]}")
print(f"Padded sequense of tokens: {[id2word[int(_)] for _ in X[2]]}")

### Подготовка данных. Dataset

Создадим класс `ToneDataset`, который наследуется от класса `torch.utils.data.Dataset`. Этот класс нужен для обработки данных.

Его методы:
- `__len__` — считает длину датасета;
- `__getitem__`:
  - возвращает по индексу текст из датасета (`tokens`);
  - преобразует текст в тензор индексов (`ids`) в соответствии со словарем;
  - возвращает по индексу тональность текста из датасета (`y`).
- `collate` — итерация по батчам:
  - разделяет список кортежей (`zip(*batch)`) на тексты (`ids`) и метки (`y`);
  - паддит последовательность до максимальной длины в батче (`pad_sequence`);
  - преобразует метки в тензор типа `float` .

In [ ]:
from torch.utils.data import Dataset

class ToneDataset(Dataset):

    def __init__(self, dataset, word2id):
        self.dataset = dataset['preprocessed'].values
        self.word2id = word2id
        self.length = dataset.shape[0]
        self.target = dataset['tone'].values

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        tokens = self.dataset[index]
        ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        y = [self.target[index]]
        return ids, y

    def collate_fn(self, batch):
        ids, y = list(zip(*batch))
        padded_ids = pad_sequence(ids, batch_first=True)
        y = torch.Tensor(y)
        return padded_ids, y

In [ ]:
dataset = ToneDataset(df, word2id)

Чтобы дальнейший код воспроизводился, зафиксируем случайные состояния:

In [ ]:
import torch
import random
import numpy as np


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


set_random_seed(42)

Разделим данные на обучающую и валидационную выборку.

In [ ]:
from torch.utils.data import random_split

lengths = [int(len(df)*0.8), int(len(df)*0.1), int(len(df)*0.1)]
train_dataset, val_dataset, test_dataset = random_split(dataset, lengths=lengths)
print(f'Train dataset length:      {len(train_dataset)}')
print(f'Validation dataset length: {len(val_dataset)}')
print(f'Test dataset length:       {len(test_dataset)}')

### Формирование батчей. DataLoader

`Dataset` возвращает по одной паре "объект — метка" за раз. При обучении моделей мы обычно хотим получать обекты в виде мини-батчей, перемешивая данные на каждой эпохе для уменьшения переобучения.

`DataLoader` — это объект, который позволяет нам получать такие мини-батчи. При инициализации он принимает в себя объект `Dataset` (или `Subset`), параметры `batch_size` (размер мини-батча) и `shuffle` (перемешивать ли данные в батчах каждую эпоху), функцию для разбиения на батчи `collate_fn`.

Другие параметры, а также значения по умолчанию можно посмотреть в документации PyTorch для класса `DataLoader` [🛠️[doc]](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset,
                              collate_fn = dataset.collate_fn,
                              batch_size=3,
                              shuffle=True)
val_dataloader = DataLoader(val_dataset,
                            collate_fn = dataset.collate_fn,
                            batch_size=3,
                            shuffle=False)
test_dataloader = DataLoader(test_dataset,
                             collate_fn = dataset.collate_fn,
                             batch_size=3,
                             shuffle=False)

Так как мы разделили данные на обучающие, валидационные и тестовые (`train_dataset`, `val_dataset` и `test_dataset`), то мы создаем также три независимых `DataLoader`-а. Один из них позволит нам получать батчи из обучающей выборки, второй — из валидационной, третий — из тестовой.

Обратите внимание на параметр `shuffle`! По умолчанию он имеет значение `False`. **Для обучения нейронной сети критически важно, чтобы во время обучения батчи обучающих данных перемешивались**. Именно таким образом мы вносим **стохастичность** в процесс градиентного спуска. Поэтому для `DataLoader`-a, который будет выдавать батчи для обучения, необходимо использовать `shuffle=True`.

В противоположность этому — `DataLoader`-ы для валидационных и тестовых данных. Эти данные данные служат для оценки качества работы модели, на них не происходит обучение и градиентный спуск. Поэтому установка здесь `shuffle=True` не имеет большого смысла.

#### Итерирование по `DataLoader`

Мы загрузили набор данных в `DataLoader`, и теперь можем проходиться по нему по мере необходимости. Каждая итерация в коде ниже будет возвращать мини-батч в виде кортежа тензоров `(samples, labels)`, содержащих `batch_size=8` объектов и меток соответственно.
Так как мы установили для `train_dataloader` параметр `shuffle=True`, когда мы пройдемся по всем батчам, данные перемешаются.

In [ ]:
# get one next batch

batch, y = next(iter(train_dataloader))

print(f"Batch size: {batch.size()[0]}\n")
print(f"Texts in batch:\n{batch}")
print(f"Size of text matrix: {batch.size()} : [batch_size, num_tokens]\n")
print(f"Targets in batch:\n{y}")
print(f"Size of target vector: {y.size()} : [batch_size, num_targets]\n")

#### Взвешенное формирование батчей для работы с дисбалансом

**По умолчанию** все объекты могут попадать в батчи **равновероятно**. При работе с несбалансированными классами можно формировать батчи так, чтобы в каждом батче примерно был баланс классов.

Это может улучшать сходимость даже в случае небольшого дисбаланса или его отсутствия, т. к. мы будем избегать шаги обучения нейросети, в которых она просто не увидела какого-то класса в силу случайных причин.


<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/batch_balancing.png" width="1000"></center>

При таком подходе требуется, чтобы объекты попадали в батчи **не равновероятно, а с определенными весами**. В PyTorch эту функциональность можно получить, используя класс `WeightedRandomSampler` [🛠️[doc]](https://pytorch.org/docs/stable/data.html#torch.utils.data.WeightedRandomSampler). Для его инициализации требуется рассчитать вес каждого класса.

Сумма весов не обязана быть равна единице, важно только соотношение между весами. Часто соотношение весов делают обратно пропорциональным количеству объектов в классах.



```
_, classes_counts = torch.unique(y, return_counts=True)  # y — tensor of labels in train set
weights_for_classes = classes_counts.max() / classes_counts
```

В конструктор `WeightedRandomSampler` требуется подать два аргумента:

- список весов для **каждого** элемента в датасете;
- количество элементов (можно использовать не весь датасет).

Затем созданный `sampler` передается в конструктор `DataLoader`.

```
from torch.utils.data import WeightedRandomSampler

weight_for_every_sample = []  # Every sample must have a weight
for label in y:
    weight_for_every_sample.append(weights_for_classes[label].item())

sampler = WeightedRandomSampler(torch.tensor(weight_for_every_sample), len(dataset))
dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)
```

Когда у `DataLoader` указывается `sampler`, последний принимает на себя функцию перемешивания, поэтому параметр `shuffle` **должен** быть не указан.

Также отметим, что подобную процедуру взвешенного формирования батчей следует производить только с обучающими данными, но не с валидационными и тестовыми. Мы хотим, чтобы модель при обучении поровну видела объекты всех классов, но оценивать качество нужно на данных с реальным распределением.

Стоит отметить, что нужно быть осторожным со взвешиванием объектов в батчах и **контролировать состав батчей**. Дело в том, что при существенном дисбалансе веса при объектах минорного класса могут оказываться на несколько порядков больше, чем при объектах мажорного класса. Данные веса преобразуются в вероятности для сэмплирования, и может случиться так, что вероятности при объектах мажорного класса станут численно неотличимы от нуля. Тем самым можно получить обратный эффект: батчи будут состоять исключительно из объектов минорного класса. В таком случае нужно намеренно ограничивать веса.

## Создание нейронной сети

Нейронные сети состоят из слоев, которые производят преобразования над данными. В PyTorch принято называть слои ***модулями*** (modules), и далее мы тоже будем использовать это название.

Пространство имен `torch.nn` [🛠️[doc]](https://pytorch.org/docs/stable/nn.html) предоставляет "строительные блоки", которые нужны для создания своей собственной нейронной сети. Каждый *модуль* в PyTorch является дочерним классом от `nn.Module` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). Таким образом, нейронная сеть сама по себе будет являться *модулем*, состоящим из других *модулей* (слоев). Такая вложенная структура позволяет легко создавать сложные архитектуры и управлять ими.

Ниже мы рассмотрим пример создания нейронной сети для классификации изображений из набора данных MNIST.

### Выбор устройства (device) для обучения

Мы бы хотели иметь возможность обучать модель на аппаратном ускорителе, таком как GPU, если он доступен. Проверим, доступен ли нам ускоритель `torch.cuda` [🛠️[doc]](https://pytorch.org/docs/stable/notes/cuda.html), иначе продолжим вычисления на CPU.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

### Описание класса модели

Мы определяем нейронную сеть, наследуясь от класса `nn.Module`, и инициализируем ее слои в методе `__init__`. Каждый класс-наследник `nn.Module` производит операции над входными данными в методе `forward`.

Напишем собственную нейронную сеть как класс `NeuralNetwork`. Ниже подробно рассмотрим все составляющие ее части.

In [ ]:
from torch import nn

class NeuralNetwork(nn.Module):

    def __init__(self, vocab_size, embedding_dim):

        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding2hidden = nn.Linear(embedding_dim, 10)
        self.act1 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.hidden2out = nn.Linear(10, 1)
        self.act2 = nn.Sigmoid()

    def forward(self, text): # seq_len
        embedded = self.embedding(text) # seq_len x embedding_dim
        mean_emb = torch.mean(embedded, dim=1) # embedding_dim
        hidden = self.embedding2hidden(mean_emb) # 10
        hidden = self.act1(hidden) # 10
        hidden = self.dropout(hidden) # 10
        out = self.hidden2out(hidden) # 1
        proba = self.act2(out) # 1
        return proba

Создадим экземпляр класса `NeuralNetwork` и выведем информацию о структуре модели.

In [ ]:
vocab_size = len(word2id)
embedding_dim = 5
model = NeuralNetwork(vocab_size, embedding_dim).to(device)
print(model)

Чтобы использовать модель, мы передаем ей входные данные. Это приводит в действие метод `forward`, а также определенные фоновые операции. Не следует вызывать `model.forward` напрямую!

Вызов модели с входными данными возвращает тензор с двумя размерностями: нулевая размерность `dim=0` соответствует  количеству переданных примеров, а первая `dim=1` — выходным предсказаниям в диапазоне $(0,1)$, т.е. вероятности отнесения примера к положительному классу.

In [ ]:
batch, y = next(iter(train_dataloader))
batch, y = batch.to(device), y.to(device)

output = model(batch)

print(f"Size of text matrix:   {batch.size()} : [batch_size, num_tokens]")
print(f"Size of target vector: {y.size()} : [batch_size, num_targets]")
print(f"Size of output vector: {output.size()} : [batch_size, num_predictions]")
print(f"Output:\n{output}")

Давайте заглянем "под капот" нашей модели `NeuralNetwork`. Для иллюстрации возьмем мини-батч и посмотрим, что с ним происходит, когда мы пропускаем его через сеть.

#### Слой `nn.Embedding`

Слой `nn.Embedding` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) преобразует входную информацию в пространство меньшей размерности, позволяя сети больше узнавать о взаимосвязи между входными данными и обрабатывать данные более эффективно.

Мы представили предложения в виде векторов из индексов. Слой эмбеддингов сопоставляет каждое слово с вектором меньшей размерности.

Создадим слой эмбеддингов с помощью метода `nn.Embedding()`. Метод принимает на вход два параметра: размер всего словаря и размер получаемого эмбеддинга.

Веса слоя эмбеддингов инициализируются случайно и будут оптимизироваться в процессе обучения (`requires_grad=True`).

In [ ]:
embedding = nn.Embedding(vocab_size, embedding_dim)
print(f"Embedding layer size: {embedding.weight.shape}: [vocab_size, embedding_dim]\n")
print(f"Embedding layer weights:\n{embedding.weight.data}")

Сейчас наши предложения представлены как векторы индексов, каждая позиция вектора соответствует одному слову.

In [ ]:
print(f"Text matrix in batch:\n{batch}")
print(f"Size of text matrix : {batch.size()} : [batch_size, num_tokens]")

 Мы передаем эти векторы на слой эмбеддингов. Теперь каждому слову соответствует не просто индекс, а его эмбеддинг. Каждому предложению в батче соответствует матрица.

In [ ]:
embedded = embedding(batch)
print(f"Size of text matrix after embedding layer:        {embedded.shape} : [batch_size, num_tokens, embedding_dim]")
print(f"Size of embedded input for 1st sentence in batch: {embedded[0,:,:].shape}    : [num_tokens, embedding_dim]")
print(f"Embedded input for 1st sentence in batch:\n{embedded[0,:,:]}")

Каждое предложение должно быть представлено в виде вектора, а не матрицы. Чтобы получить эмбеддинг предложения, можно усреднить эмбеддинг каждого слова.

In [ ]:
mean_emb = torch.mean(embedded, dim=1)
print(f"Size of text matrix after averaging: {mean_emb.shape} : [batch_size, num_tokens]")
print(f"Mean embedded input:\n{mean_emb}")

#### Слой `nn.Linear`

Линейный слой `nn.Linear` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) — это модуль, который производит линейное преобразование входных данных с помощью хранящихся в нем весов и смещений.

Обязательными параметрами при объявлении этого слоя являются `in_features` — количество входных признаков, и `out_features` — количество выходных признаков.

Фактически этот модуль добавляет в модель один полносвязный слой нейронов *без активаций*. Слой состоит из `out_features` нейронов, каждый из которых имеет `in_features` входов.

В примере ниже мы объявляем слой из $10$ нейронов, каждый из которых получает эмбеддинг размера $5$.

In [ ]:
embedding2hidden = nn.Linear(embedding_dim, 10)
hidden = embedding2hidden(mean_emb)
print(f"Input size:         {mean_emb.size()} : [batch_size, num_tokens]")
print(f"Size after Linear:  {hidden.size()}: [batch_size, out_features]")

Линейный слой также имеет обучаемые параметры — веса и смещения. Они хранятся как объекты специального класса `torch.nn.parameter.Parameter` и содержат в себе тензоры собственно с величинами параметров. Получить доступ к ним можно, обратившись к атрибутам слоя `.weight` и `.bias` соответственно.

In [ ]:
print(f"Size of linear layer weights: {embedding2hidden.weight.size()}")
print(f"Type of linear layer weights: {type(embedding2hidden.weight)}")

print(f"\nSize of linear layer biases: {embedding2hidden.bias.size()}")
print(f"Type of linear layer biases: {type(embedding2hidden.bias)}")

#### Слой `nn.ReLU`

Нелинейные активации — это то, что позволяет модели учить сложные взаимосвязи между входом и выходом. Они применяются после линейных преобразований, чтобы ввести *нелинейность*, помогая нейронным сетям изучать самые разные закономерности.

В данной модели мы используем `nn.Relu` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) между линейными слоями, но существуют и реализации [других функций активации 🛠️[doc]](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity).

In [ ]:
act1 = nn.ReLU()(hidden)

print(f"Vector for 1st sentence before ReLU:\n{hidden[0,:]}\n")
print(f"Vector for 1st sentence after ReLU:\n{act1[0,:]}\n")
print(f"Size before ReLU:  {hidden.size()} : [batch_size, num_features]")
print(f"Size after ReLU:   {act1.size()} : [batch_size, num_features]")

#### Слой `nn.Dropout`

`nn.Dropout` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html) — операция, которая позволяет избежать переобучения сети. При обучении некоторые признаки могут стать слишком "сильными", они слишком подстраиваются под обучающую выборку. При каждом прохождении через сеть некоторые случайно выбранные признаки "выпадают" (drop out), т.е. приравниваются к нулю, сигнал с них не будет проходить дальше.

В качестве параметра слой дропаут принимает вероятность, с которой признаки зануляются. Вероятность является гиперпараметром и устанавливается самостоятельно.

In [ ]:
dropout = nn.Dropout(p=0.5)(act1)
print(f"Vector for 1st sentence before Dropout:\n{act1[0,:]}\n")
print(f"Vector for 1st sentence after Dropout:\n{dropout[0,:]}\n")
print(f"Size before Dropout:  {act1.size()} : [batch_size, num_features]")
print(f"Size after Dropout:   {dropout.size()} : [batch_size, num_features]")

Зануление части признаков не является обязательным. Но если вы видите, что происходит переобучение, можно добавить дропаут и поэкспериментировать со значениями (обычно от 0.1 до 0.5).

Обратите внимание, что `dropout` должен применяться только при обучении модели. При тестировании веса не зануляются.

Чтобы контролировать это, необходимо при обучении переводить модель в состояние обучения:

`model.eval()`,

а при применении — в состояние тестирования:

`model.eval()`.

#### Слой `nn.Linear`

Второй линейный слой получает выход первого слоя длины $10$ и отдает $1$ значение.

In [ ]:
hidden2out = nn.Linear(10, 1)
logits = hidden2out(dropout)
print(f"Input size:         {dropout.size()} : [batch_size, in_features]")
print(f"Size after Linear:  {logits.size()}:   [batch_size, out_features]")

#### Слой `nn.Sigmoid()`

Последний линейный слой нейронной сети возвращает *логиты* — "сырые" значения из диапазона $[-∞; +∞]$, которые пропускаются через модуль `nn.Sigmoid()` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html). Пропущенные через $\text{Sigmoid}$ величины могут восприниматься как вероятности, с которыми модель относит данный объект к тому или иному классу.

In [ ]:
act2 = nn.Sigmoid()

pred_probab = act2(logits)

print(f"Logits:\n{logits}\n")
print(f"Predicted probabilities:\n{pred_probab}\n")
print(f"Size before Sigmoid: {logits.size()} : [batch_size, num_predictions]")
print(f"Size after Sigmoid:  {pred_probab.size()} : [batch_size, num_predictions]")

### Параметры модели

Множество слоев в нейронных сетях имеют *обучаемые параметры*, т. е. имеют ассоциированные с ними веса и смещения, которые оптимизируются во время обучения.

Наследование от `nn.Module` автоматически отслеживает все слои, определенные внутри вашего класса модели, и делает все их параметры доступными с помощью методов `model.parameters()` или `model.named_parameters()`.

В примере ниже мы проходимся по всем параметрам модели и для каждого тензора параметров выводим его размер.

In [ ]:
print(f"Model structure: {model}\n")

for name, param in model.named_parameters():
    print(
        f"Layer: {name:23}  Requires grad: {param.requires_grad}  Size: {param.size()}"
    )

## Обучение нейронной сети

Теперь, когда мы подготовили данные и создали модель, мы можем приступить к обучению.

Обучение модели — итеративный процесс оптимизации параметров модели на обучающих данных. На каждой итерции модель получает входные данные, дает предсказание на выходе, вычисляет значение функции потерь (loss), вычисляет производные функции потерь по параметрам и подстраивает параметры, используя градиентный спуск.

### Гиперпараметры

Гиперпараметры — это задаваемые разработчиком параметры, которые позволяют управлять процессом обучения. Различные значения гиперпараметров могут влиять на обучение модели и скорость сходимости ([подробнее про подбор гиперпараметров 🛠️[doc]](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)).

Мы определим следующие гиперпараметры процедуры обучения:
* **количество эпох** (`num_epochs`) — количество итераций обучения по всему набору данных;
* **размер батча** (`batch_size`) — количество образцов, передаваемых в сеть для обновления параметров;
* **скорость обучения** (`learning_rate`) — коэффициент, определяющий, насколько сильно нужно обновлять параметры модели на каждом батче. Малые значения приводят к долгому обучению, в то время как большие значения могут приводить к непредсказуемому поведению во время обучения.


In [ ]:
num_epochs = 15
batch_size = 256
learning_rate = 0.001

Выше для демонстрации мы указывали размер батча в `DataLoader`-ах равным восьми. Для установки нового значения придется переопределить `DataLoader`-ы.

In [ ]:
train_dataloader = DataLoader(train_dataset,
                              collate_fn=dataset.collate_fn,
                              batch_size=batch_size,
                              shuffle=True)
val_dataloader = DataLoader(val_dataset,
                            collate_fn=dataset.collate_fn,
                            batch_size=batch_size,
                            shuffle=False)
test_dataloader = DataLoader(test_dataset,
                             collate_fn=dataset.collate_fn,
                             batch_size=batch_size,
                             shuffle=False)

### Оптимизация параметров (обучение сети)

Задав гиперпараметры, мы можем приступить к обучению модели. Каждая итерация цикла оптимизации называется **эпохой**.

Каждая эпоха состоит из двух частей:
* **цикл обучения (Train Loop)** — проход по обучающему набору данных и оптимизация параметров;
* **цикл валидации (Validation Loop)** — проход по валидационному набору данных и контроль того, что качество работы сети улучшается.

Кратко ознакомимся с некоторыми понятиями, используемыми в цикле оптимизации.

#### Функция потерь (Loss function)

Так как мы решаем задачу бинарной классификации, выберем `nn.BCELoss` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html) в качестве функции потерь.

In [ ]:
# Initialize the loss function
criterion = nn.BCELoss()

#### Оптимизатор (Optimizer)

Оптимизация — это процесс подстройки параметров модели для уменьшения ошибки на каждом шаге обучения. От **алгоритма оптимизации** зависит то, как этот процесс будет выполняться. Здесь мы будем использовать Adam. Однако в PyTorch реализовано еще [множество других алгоритмов оптимизации 🛠️[doc]](https://pytorch.org/docs/stable/optim.html#algorithms), таких как SGD и RMSProp, и они могут работать лучше или хуже для разных видов моделей и данных.

Вся логика оптимизации заключена в объекте `optimizer`. Мы инициализируем оптимизатор, передавая ему параметры модели, которые требуется обучать (`model.parameters()`), а также гиперпараметр скорости обучения (`learning_rate`).

In [ ]:
model = NeuralNetwork(vocab_size, embedding_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Внутри цикла обучения оптимизация производится за три шага:
1. Вызов `optimizer.zero_grad()`, чтобы сбросить градиенты параметров модели. По умолчанию градиенты суммируются, и во избежание повторного вычисления их необходимо явно обнулять на каждой итерации;
2. Обратное распространение ошибки предсказания с помощью вызова `loss.backward()`. PyTorch вычислит градиенты функции потерь относительно каждого обучаемого параметра;
3. Когда у нас есть градиенты, мы вызываем `optimizer.step()`, чтобы подстроить обучаемые параметры с учетом градиентов, посчитанных при обратном распространении, согласно алгоритму оптимизации.

### Реализация обучения

Мы определим две функции:
* `train_loop`, которая производит цикл обучения,
* `val_loop`, которая оценивает качество модели на валидационных данных.

In [ ]:
def train_loop(dataloader, model, criterion, optimizer):

    num_batches = len(dataloader)
    train_loss = 0
    model.train()

    for texts, labels in dataloader:
        # Compute prediction and loss
        pred = model(texts.to(device))
        loss = criterion(pred, labels.to(device))

        # Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.detach().item()

    train_loss /= num_batches
    print(f"Train loss: {train_loss:>8f}")

    return train_loss


def val_loop(dataloader, model, criterion):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss, correct = 0, 0
    model.eval()

    with torch.no_grad():
        for texts, labels in dataloader:
            # Compute prediction and loss
            pred = model(texts.to(device))
            loss = criterion(pred, labels.to(device))

            val_loss += loss.item()
            correct += ((pred>0.5).int() == labels.to(device)).sum().item()

    val_loss /= num_batches
    accuracy = correct / size
    print(f"Val loss: {val_loss:>8f}, val accuracy: {(100*accuracy):>0.1f}% \n")

    return val_loss

Выше мы объявили гиперпараметры и инициализировали функцию потерь `criterion` и оптимизатор `optimizer`. Теперь мы запускаем цикл оптимизации на $10$ эпох, и в каждой итерации мы вызываем функцию для выполнения цикла обучения `train_loop`, а затем функцию для промежуточной оценки качества `val_loop`. Также на каждой эпохе будем сохранять текущее значение функции потерь на обучающих и валидационных данных для построения графика обучения.

In [ ]:
# for plotting
loss_history = {"train": [], "val": []}

for i in range(num_epochs):
    print(f"Epoch {i+1}")
    train_loss = train_loop(train_dataloader, model, criterion, optimizer)
    val_loss = val_loop(val_dataloader, model, criterion)

    loss_history["train"].append(train_loss)
    loss_history["val"].append(val_loss)
print("Done!")

Построим график функции потерь на обучающих и на валидационных данных по эпохам:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), loss_history["train"], label="train")
plt.plot(range(1, num_epochs + 1), loss_history["val"], label="val")
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Loss", fontsize=15)
plt.legend()
plt.grid()
plt.show()

### Сохранение и загрузка весов модели

Обычно обучение модели является достаточно затратным процессом с точки зрения вычислительных ресурсов. Поэтому однажды обученную сеть разумно будет сохранить для последующего использования. Рассмотрим варианты, как это можно сделать в PyTorch.

Модели PyTorch хранят обучаемые параметры во внутреннем словаре состояния, который называется `state_dict`. Их можно сохранить с помощью метода `torch.save`.


In [ ]:
torch.save(model.state_dict(), "model_weights.pth")

Для того, чтобы загрузить веса модели, сперва необходимо создать экземпляр такой модели, а затем загрузить параметры с помощью метода `load_state_dict()`.

In [ ]:
model = NeuralNetwork(vocab_size, embedding_dim)
model.load_state_dict(torch.load("model_weights.pth"))
model.to(device)
print(model)

Данный способ сохранения модели подходит только для случая, когда в дальнейшем не предполагается продолжать обучать модель. Для продолжения обучения также необходимо как минимум сохранять состояние оптимизатора.

[[doc] 🛠️ О более общем сохранении состояний модели, оптимизатора и других объектов](https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-a-general-checkpoint-for-inference-and-or-resuming-training)

Сохранение в общем случае может выглядеть так:

```
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss_history': loss_history,
            ...
            }, PATH)
```



###  Предсказания обученной модели

Пропустим тестовые тексты через модель и визуализируем результат.

In [ ]:
size = len(test_dataloader.dataset)
val_loss, correct = 0, 0
model.eval()

with torch.no_grad():
    for texts, labels in test_dataloader:
        # Compute prediction
        pred = model(texts.to(device))
        correct += ((pred>0.5).int() == labels.to(device)).sum().item()

    accuracy = correct / size
    print(f"Test accuracy: {(100*accuracy):>0.1f}%")

In [ ]:
indexes = {8: 31068, 10 : 174001, 24: 63690, 40: 180925}
for key, value in indexes.items():
  print(f"Sentence: {df['text'][value]}\n\
  tone: {labels[key].item()},\n\
  prediction: {(pred[key]>0.5).int().item()}\n")

<font size=6>Литература</font>

<font size=5>Многослойные нейронные сети:</font>

* [[demo] 🎮 Интерактивный тренажер от TensorFlow ](http://playground.tensorflow.org/#activation=linear&batchSize=10&dataset=xor&regDataset=reg-plane&learningRate=0.1&regularizationRate=0&noise=0&networkShape=&seed=0.62952&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false)
* [[wiki] 📚 Теорема Цыбенко](https://ru.wikipedia.org/wiki/Теорема_Цыбенко)

<font size=5>Обучение нейронной сети:</font>

* [[blog] ✏️ ML: Вычислительный граф](https://qudata.com/ml/ru/ML_Comp_Graph.html)
* [[blog] ✏️ Обзор функций потерь в PyTorch с примером написания своей собственной функции (custom loss function)](https://neptune.ai/blog/pytorch-loss-functions)
* [[blog] ✏️ Объяснение Negative Log Likelihood Loss](https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/)
* [[blog] ✏️ О соотношении Cross-Entropy Loss и Negative Log Likelihood Loss](https://jamesmccaffrey.wordpress.com/2020/06/11/pytorch-crossentropyloss-vs-nllloss-cross-entropy-loss-vs-negative-log-likelihood-loss/)
* [[blog] ✏️ How Activation Functions Work in Deep Learning](https://www.kdnuggets.com/2022/06/activation-functions-work-deep-learning.html)

<font size=5>Углубление в PyTorch. Пример нейронной сети:</font>
* [[video] 📺 Плей-лист с объяснением базовых принципов нейронных сетей от 3Blue1Brown](https://youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&si=O3gq6RoIxoNS6iJf) [[озвучка на русском](https://youtube.com/playlist?list=PLZjXXN70PH5itkSPe6LTS-yPyl5soOovc&si=ElKVi98Ui4m2nJKq)]
* [[doc] 🛠️ PyTorch: Сохранение состояний модели, оптимизатора и других объектов](https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-a-general-checkpoint-for-inference-and-or-resuming-training)

<font size=5>Дополнительно:</font>
* [[book] 📚 Deep Learning](https://www.deeplearningbook.org/)
 * [[book] 📚 Part 2, Chapter 6: Deep Feedforward Networks](https://www.deeplearningbook.org/contents/mlp.html)
* [[demo] 🎮 Интерактивное и визуальное объяснение работы многослойных нейронных сетей](https://mlu-explain.github.io/neural-networks/)
* [[demo] 🎮 Интерактивная визуализация ландшафтов функций потерь в нейронных сетях](https://losslandscape.com/explorer)
* [[blog] ✏️ The 7 Most Common Machine Learning Loss Functions](https://builtin.com/machine-learning/common-loss-functions)
* [[blog] ✏️ Understanding Categorical Cross-Entropy Loss, Binary Cross-Entropy Loss, Softmax Loss, Logistic Loss, Focal Loss and all those confusing names](https://gombru.github.io/2018/05/23/cross_entropy_loss/)
* Курс А. Г. Дьяконова "Глубокое обучение". Лекции по PyTorch:
 * [[video] 📺 часть 1](https://www.youtube.com/watch?v=tDJnwc8Hioc)
 * [[video] 📺 часть 2](https://www.youtube.com/watch?v=c3y--ydWku0)